In [2]:
# 필수 라이브러리 임포트
import torch
import warnings
import os
import pickle
import json
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity

# 설정
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 사용 디바이스: {device}")

# 시드 설정
torch.manual_seed(42)
np.random.seed(42)

# 한글 폰트 설정 (필요시 수정)
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False

print("✅ 환경 설정 완료")

🚀 사용 디바이스: cpu
✅ 환경 설정 완료


In [3]:
# In[2]:
class MusicEmotionClassifier(nn.Module):
    """음악 감정 분류를 위한 신경망 모델"""
    
    def __init__(self, input_dim, hidden_layers=[256, 128, 64], num_classes=10, dropout=0.3):
        super(MusicEmotionClassifier, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        # 히든 레이어 구성
        for i, hidden_dim in enumerate(hidden_layers):
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout if i < len(hidden_layers)-1 else dropout/2)
            ])
            prev_dim = hidden_dim
        
        # 출력 레이어
        layers.extend([
            nn.Linear(prev_dim, num_classes),
            nn.Softmax(dim=1)
        ])
        
        self.network = nn.Sequential(*layers)
        self._initialize_weights()
    
    def _initialize_weights(self):
        """가중치 초기화"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                nn.init.zeros_(module.bias)
    
    def forward(self, x):
        return self.network(x)

print("✅ MusicEmotionClassifier 모델 정의 완료")



✅ MusicEmotionClassifier 모델 정의 완료


In [4]:
# ## 3. 감정 매핑 및 분류 규칙 정의

# In[3]:
class EmotionConfig:
    """감정 설정 및 매핑 클래스"""
    
    def __init__(self):
        # 감정 카테고리 정의 (인사이드아웃 2 기반)
        self.emotions = {
            'joy': {'valence': (0.8, 1.0), 'energy_weight': 0.7, 'label': '기쁨이'},
            'sadness': {'valence': (0.0, 0.25), 'energy_weight': 0.2, 'label': '슬픔이'},
            'boredom': {'valence': (0.4, 0.6), 'energy_weight': 0.1, 'label': '따분이'},
            'anxiety': {'valence': (0.1, 0.4), 'energy_weight': 0.5, 'label': '소심이'},
            'anger': {'valence': (0.0, 0.3), 'energy_weight': 0.8, 'label': '버럭이'},
            'grumpy': {'valence': (0.3, 0.5), 'energy_weight': 0.5, 'label': '까칠이'},
            'fear': {'valence': (0.0, 0.2), 'energy_weight': 0.9, 'label': '불안이'},
            'embarrassment': {'valence': (0.5, 0.75), 'energy_weight': 0.3, 'label': '당황이'},
            'envy': {'valence': (0.2, 0.4), 'energy_weight': 0.4, 'label': '부럽이'},
            'calm': {'valence': (0.4, 0.75), 'energy_weight': 0.3, 'label': '평온'},
        }
        
        # 한국어 감정 키워드 매핑
        self.korean_emotion_map = {
            # 기쁨 관련
            '기쁨': 'joy', '환희': 'joy', '즐거움': 'joy', '행복': 'joy', '만족': 'joy',
            '신나는': 'joy', '기쁜': 'joy', '즐거운': 'joy', '행복한': 'joy', '좋은': 'joy',
            '설렘': 'joy', '상쾌': 'joy', '활기': 'joy', '최고': 'joy', '완벽': 'joy',
            
            # 슬픔 관련  
            '슬픔': 'sadness', '우울': 'sadness', '좌절': 'sadness', '비참': 'sadness',
            '슬픈': 'sadness', '우울한': 'sadness', '외로운': 'sadness', '암울': 'sadness',
            '그리움': 'sadness', '아쉬움': 'sadness', '후회': 'sadness', '실망': 'sadness',
            
            # 지루함 관련
            '지루함': 'boredom', '따분': 'boredom', '심심': 'boredom', '무기력': 'boredom',
            '지루한': 'boredom', '심심한': 'boredom', '따분한': 'boredom', '재미없': 'boredom',
            
            # 불안 관련
            '불안': 'anxiety', '걱정': 'anxiety', '초조': 'anxiety', '긴장': 'anxiety',
            '스트레스': 'anxiety', '답답': 'anxiety', '불안한': 'anxiety', '걱정되는': 'anxiety',
            
            # 분노 관련
            '화남': 'anger', '분노': 'anger', '짜증': 'anger', '버럭': 'anger',
            '화난': 'anger', '짜증나는': 'anger', '열받': 'anger', '빡친': 'anger',
            
            # 까칠함 관련
            '까칠': 'grumpy', '예민': 'grumpy', '불쾌': 'grumpy',
            
            # 두려움 관련
            '두려움': 'fear', '공포': 'fear', '무서움': 'fear', '겁': 'fear',
            '무서운': 'fear', '두려운': 'fear', '겁나는': 'fear',
            
            # 당황 관련
            '당황': 'embarrassment', '어색': 'embarrassment', '민망': 'embarrassment',
            
            # 부러움 관련
            '부러움': 'envy', '질투': 'envy',
            
            # 평온 관련
            '평온': 'calm', '차분': 'calm', '안정': 'calm', '편안': 'calm',
            '여유': 'calm', '느긋': 'calm', '평온한': 'calm', '차분한': 'calm',
        }
    
    def classify_emotion(self, valence, energy):
        """Valence-Energy 값으로 감정 분류"""
        # 감정 분류 로직 (순서 중요)
        if valence >= 0.8 and energy >= 0.7:
            return 'joy'
        elif valence < 0.2 and energy >= 0.85:
            return 'fear'
        elif valence < 0.3 and energy >= 0.7:
            return 'anger'
        elif 0.2 <= valence < 0.4 and energy >= 0.4:
            return 'envy'
        elif 0.1 <= valence < 0.4 and energy >= 0.5:
            return 'anxiety'
        elif valence < 0.25 and energy < 0.3:
            return 'sadness'
        elif 0.4 <= valence < 0.6 and energy < 0.2:
            return 'boredom'
        elif 0.3 <= valence < 0.5 and energy >= 0.5:
            return 'grumpy'
        elif 0.5 <= valence < 0.75 and energy < 0.4:
            return 'embarrassment'
        else:
            return 'calm'
    
    def get_emotion_label(self, emotion_key):
        """감정 키에 대한 한국어 라벨 반환"""
        return self.emotions.get(emotion_key, {'label': '알 수 없음'})['label']
    
    def map_korean_emotion(self, korean_text):
        """한국어 텍스트에서 감정 매핑"""
        for keyword, emotion in self.korean_emotion_map.items():
            if keyword in korean_text:
                return emotion, keyword
        return None, None

# 전역 설정 객체 생성
emotion_config = EmotionConfig()
print("✅ 감정 설정 완료")

✅ 감정 설정 완료


In [5]:
# ## 4. 데이터 로딩 및 전처리 함수들

# In[4]:
class DataLoader:
    """데이터 로딩 및 전처리 클래스"""
    
    def __init__(self, emotion_config):
        self.emotion_config = emotion_config
        self.train_df = None
        self.test_df = None
    
    def load_train_data(self, file_path):
        """학습 데이터 로드 (Spotify Tracks Dataset)"""
        try:
            if not os.path.exists(file_path):
                print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
                return False
            
            print(f"📂 학습 데이터 로딩 중: {file_path}")
            self.train_df = pd.read_csv(file_path)
            
            # 컬럼명 정리
            self.train_df.columns = self.train_df.columns.str.strip()
            
            print(f"📊 원본 데이터: {len(self.train_df):,} 트랙")
            
            # 필수 컬럼 확인
            required_cols = ['valence', 'energy', 'danceability', 'acousticness']
            missing_cols = [col for col in required_cols if col not in self.train_df.columns]
            
            if missing_cols:
                print(f"❌ 필수 컬럼 누락: {missing_cols}")
                return False
            
            # 데이터 전처리
            self._preprocess_train_data()
            
            print(f"✅ 전처리 완료: {len(self.train_df):,} 트랙")
            self._print_emotion_distribution(self.train_df, "학습")
            
            return True
            
        except Exception as e:
            print(f"❌ 데이터 로딩 오류: {e}")
            return False
    
    def load_test_data(self, file_path):
        """테스트 데이터 로드 (Spotify YouTube Dataset)"""
        try:
            if not os.path.exists(file_path):
                print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
                return False
            
            print(f"📂 테스트 데이터 로딩 중: {file_path}")
            self.test_df = pd.read_csv(file_path)
            
            # 컬럼명 정리 및 매핑
            self.test_df.columns = self.test_df.columns.str.strip()
            self._map_test_columns()
            
            print(f"📊 원본 데이터: {len(self.test_df):,} 트랙")
            
            # 데이터 전처리
            self._preprocess_test_data()
            
            print(f"✅ 전처리 완료: {len(self.test_df):,} 트랙")
            self._print_emotion_distribution(self.test_df, "테스트")
            
            return True
            
        except Exception as e:
            print(f"❌ 데이터 로딩 오류: {e}")
            return False
    
    def _preprocess_train_data(self):
        """학습 데이터 전처리"""
        # 필수 컬럼 결측값 제거
        required_cols = ['valence', 'energy', 'danceability', 'acousticness']
        self.train_df.dropna(subset=required_cols, inplace=True)
        
        # 0~1 범위 밖의 값 제거
        for col in required_cols:
            self.train_df = self.train_df[
                (self.train_df[col] >= 0) & (self.train_df[col] <= 1)
            ]
        
        # 감정 분류 적용
        self.train_df['emotion_cat'] = self.train_df.apply(
            lambda row: self.emotion_config.classify_emotion(row['valence'], row['energy']), 
            axis=1
        )
    
    def _preprocess_test_data(self):
        """테스트 데이터 전처리"""
        # 사용 가능한 특성 확인
        audio_features = ['valence', 'energy', 'danceability', 'acousticness', 
                         'instrumentalness', 'liveness', 'speechiness', 'loudness', 'tempo']
        available_features = [f for f in audio_features if f in self.test_df.columns]
        
        # 결측값 제거
        self.test_df.dropna(subset=available_features, inplace=True)
        
        # 0~1 범위 정규화 (해당하는 특성들만)
        normalized_features = ['valence', 'energy', 'danceability', 'acousticness']
        for col in normalized_features:
            if col in self.test_df.columns:
                self.test_df[col] = pd.to_numeric(self.test_df[col], errors='coerce')
                self.test_df = self.test_df[
                    (self.test_df[col] >= 0) & (self.test_df[col] <= 1)
                ].copy()
        
        # 다시 결측값 제거
        self.test_df.dropna(subset=available_features, inplace=True)
        
        # 감정 분류 적용
        if 'valence' in self.test_df.columns and 'energy' in self.test_df.columns:
            self.test_df['emotion_cat'] = self.test_df.apply(
                lambda row: self.emotion_config.classify_emotion(row['valence'], row['energy']), 
                axis=1
            )
        else:
            self.test_df['emotion_cat'] = 'calm'
    
    def _map_test_columns(self):
        """테스트 데이터 컬럼명 매핑"""
        column_mapping = {
            'Track': 'track_name',
            'Artist': 'track_artist',
            'Danceability': 'danceability',
            'Energy': 'energy',
            'Valence': 'valence',
            'Acousticness': 'acousticness',
            'Instrumentalness': 'instrumentalness',
            'Liveness': 'liveness',
            'Speechiness': 'speechiness',
            'Loudness': 'loudness',
            'Tempo': 'tempo'
        }
        
        for old_name, new_name in column_mapping.items():
            if old_name in self.test_df.columns:
                self.test_df.rename(columns={old_name: new_name}, inplace=True)
    
    def _print_emotion_distribution(self, df, data_type):
        """감정 분포 출력"""
        if 'emotion_cat' in df.columns:
            print(f"📈 {data_type} 데이터 감정 분포:")
            emotion_counts = df['emotion_cat'].value_counts()
            for emotion, count in emotion_counts.items():
                label = self.emotion_config.get_emotion_label(emotion)
                print(f"   - {label}: {count:,} 트랙")
    
    def get_common_features(self):
        """학습/테스트 데이터의 공통 특성 반환"""
        if self.train_df is None or self.test_df is None:
            return []
        
        standard_features = ['valence', 'energy', 'danceability', 'acousticness',
                           'instrumentalness', 'liveness', 'speechiness', 'loudness', 'tempo']
        
        common_features = list(
            set(standard_features) & 
            set(self.train_df.columns) & 
            set(self.test_df.columns)
        )
        
        return common_features

# 데이터 로더 인스턴스 생성
data_loader = DataLoader(emotion_config)
print("✅ 데이터 로더 준비 완료")


✅ 데이터 로더 준비 완료


In [10]:
# ## 5. 데이터 로드 실행

# In[5]:
# 데이터 파일 경로 설정 
TRAIN_DATA_PATH = r"C:\Users\USER\OneDrive\문서\GitHub\-\dataset\Spotify_Tracks_Dataset.csv"
TEST_DATA_PATH = r"C:\Users\USER\OneDrive\문서\GitHub\-\dataset\Spotify_Youtube.csv"

print("📂 데이터 로딩 시작...")
print(f"학습 데이터: {TRAIN_DATA_PATH}")
print(f"테스트 데이터: {TEST_DATA_PATH}")

# 실제 실행 (파일이 존재할 때)
train_loaded = data_loader.load_train_data(TRAIN_DATA_PATH)
test_loaded = data_loader.load_test_data(TEST_DATA_PATH)

if train_loaded and test_loaded:
    print("✅ 모든 데이터 로딩 완료")
    common_features = data_loader.get_common_features()
    print(f"🔧 공통 특성 ({len(common_features)}개): {common_features}")
else:
    print("⚠️ 데이터 파일을 확인하고 경로를 수정해주세요")

📂 데이터 로딩 시작...
학습 데이터: C:\Users\USER\OneDrive\문서\GitHub\-\dataset\Spotify_Tracks_Dataset.csv
테스트 데이터: C:\Users\USER\OneDrive\문서\GitHub\-\dataset\Spotify_Youtube.csv
📂 학습 데이터 로딩 중: C:\Users\USER\OneDrive\문서\GitHub\-\dataset\Spotify_Tracks_Dataset.csv
📊 원본 데이터: 114,000 트랙
✅ 전처리 완료: 114,000 트랙
📈 학습 데이터 감정 분포:
   - 평온: 52,158 트랙
   - 부럽이: 16,053 트랙
   - 까칠이: 9,935 트랙
   - 기쁨이: 9,624 트랙
   - 버럭이: 7,494 트랙
   - 슬픔이: 7,240 트랙
   - 불안이: 5,133 트랙
   - 당황이: 3,055 트랙
   - 소심이: 2,324 트랙
   - 따분이: 984 트랙
📂 테스트 데이터 로딩 중: C:\Users\USER\OneDrive\문서\GitHub\-\dataset\Spotify_Youtube.csv
📊 원본 데이터: 20,718 트랙
✅ 전처리 완료: 20,716 트랙
📈 테스트 데이터 감정 분포:
   - 평온: 10,871 트랙
   - 부럽이: 3,069 트랙
   - 까칠이: 2,015 트랙
   - 기쁨이: 2,001 트랙
   - 슬픔이: 892 트랙
   - 버럭이: 710 트랙
   - 당황이: 436 트랙
   - 소심이: 429 트랙
   - 불안이: 200 트랙
   - 따분이: 93 트랙
✅ 모든 데이터 로딩 완료
🔧 공통 특성 (9개): ['tempo', 'valence', 'acousticness', 'loudness', 'speechiness', 'danceability', 'liveness', 'instrumentalness', 'energy']


In [19]:
 ## 6. 데이터 전처리 및 준비

# In[6]:
class DataPreprocessor:
    """데이터 전처리 클래스"""
    
    def __init__(self):
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.imputation_means = None
        self.common_features = None
    
    def prepare_data(self, train_df, test_df, common_features):
        """학습/테스트 데이터 준비"""
        print("🔧 데이터 전처리 시작...")
        
        self.common_features = common_features
        
        # 데이터 필터링
        train_cols = common_features + ['emotion_cat']
        test_cols = common_features + ['emotion_cat']
        
        train_filtered = train_df[train_cols].copy()
        test_filtered = test_df[test_cols].copy()
        
        # 무한값 처리
        train_filtered.replace([np.inf, -np.inf], np.nan, inplace=True)
        test_filtered.replace([np.inf, -np.inf], np.nan, inplace=True)
        
        # 결측값 처리 (학습 데이터 기준으로 평균값 계산)
        self.imputation_means = train_filtered[common_features].mean()
        
        train_clean = train_filtered.fillna(self.imputation_means)
        test_clean = test_filtered.fillna(self.imputation_means)
        
        # 특성과 라벨 분리
        X_train = train_clean[common_features].values
        y_train = self.label_encoder.fit_transform(train_clean['emotion_cat'])
        
        X_test = test_clean[common_features].values
        y_test = self.label_encoder.transform(test_clean['emotion_cat'])
        
        # 스케일링
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        print(f"📊 전처리 결과:")
        print(f"   학습 데이터: {X_train_scaled.shape[0]:,} 샘플, {X_train_scaled.shape[1]} 특성")
        print(f"   테스트 데이터: {X_test_scaled.shape[0]:,} 샘플, {X_test_scaled.shape[1]} 특성")
        print(f"   감정 클래스: {list(self.label_encoder.classes_)}")
        
        return X_train_scaled, y_train, X_test_scaled, y_test
    
    def get_feature_names(self):
        """사용된 특성명 반환"""
        return self.common_features if self.common_features else []
    
    def get_emotion_classes(self):
        """감정 클래스 반환"""
        if hasattr(self.label_encoder, 'classes_'):
            return list(self.label_encoder.classes_)
        return []

# 전처리기 인스턴스 생성
preprocessor = DataPreprocessor()

# 데이터가 로드되었을 때만 전처리 실행
if hasattr(data_loader, 'train_df') and data_loader.train_df is not None:
    common_features = data_loader.get_common_features()
    if common_features:
        X_train, y_train, X_test, y_test = preprocessor.prepare_data(
            data_loader.train_df, 
            data_loader.test_df, 
            common_features
        )
        print("✅ 데이터 전처리 완료")
    else:
        print("❌ 공통 특성을 찾을 수 없습니다")
else:
    print("⚠️ 먼저 데이터를 로드해주세요")

🔧 데이터 전처리 시작...
📊 전처리 결과:
   학습 데이터: 114,000 샘플, 9 특성
   테스트 데이터: 20,716 샘플, 9 특성
   감정 클래스: ['anger', 'anxiety', 'boredom', 'calm', 'embarrassment', 'envy', 'fear', 'grumpy', 'joy', 'sadness']
✅ 데이터 전처리 완료


In [ ]:
# ## 7. 모델 학습 (수정된 버전)

# In[7]:
class ModelTrainer:
    """모델 학습 클래스"""
    
    def __init__(self, emotion_config, device):
        self.emotion_config = emotion_config
        self.device = device
        self.model = None
        self.training_history = []
    
    def train_model(self, X_train, y_train, X_test, y_test, 
                   epochs=100, batch_size=256, learning_rate=1e-3):
        """모델 학습"""
        print("🚀 모델 학습 시작...")
        
        # 텐서 변환
        X_train_tensor = torch.from_numpy(X_train).float()
        y_train_tensor = torch.from_numpy(y_train).long()
        X_test_tensor = torch.from_numpy(X_test).float()
        y_test_tensor = torch.from_numpy(y_test).long()
        
        # PyTorch DataLoader를 명시적으로 사용 (충돌 방지)
        from torch.utils.data import DataLoader as TorchDataLoader
        
        # 데이터 로더 생성
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_dataloader = TorchDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        # 모델 생성
        input_dim = X_train.shape[1]
        num_classes = len(np.unique(y_train))
        
        self.model = MusicEmotionClassifier(
            input_dim=input_dim,
            num_classes=num_classes
        ).to(self.device)
        
        # 옵티마이저와 손실함수
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=1e-4)
        criterion = nn.CrossEntropyLoss()
        
        print(f"📈 학습 시작 - {epochs} 에포크, 배치 크기: {batch_size}")
        
        # 학습 루프
        self.training_history = []
        
        for epoch in tqdm(range(epochs), desc="학습 진행"):
            self.model.train()
            epoch_loss = 0.0
            correct_predictions = 0
            total_samples = 0
            
            for batch_X, batch_y in train_dataloader:
                batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
                
                # 순전파
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_y)
                
                # 역전파
                loss.backward()
                optimizer.step()
                
                # 통계 업데이트
                epoch_loss += loss.item()
                predicted = outputs.argmax(1)
                correct_predictions += (predicted == batch_y).sum().item()
                total_samples += batch_y.size(0)
            
            # 🔧 수정된 부분: train_loader -> train_dataloader
            avg_loss = epoch_loss / len(train_dataloader)
            accuracy = correct_predictions / total_samples
            
            self.training_history.append({
                'epoch': epoch + 1,
                'loss': avg_loss,
                'accuracy': accuracy
            })
            
            # 진행 상황 출력
            if (epoch + 1) % 20 == 0:
                print(f"에포크 {epoch+1}/{epochs} - 손실: {avg_loss:.4f}, 정확도: {accuracy:.3f}")
        
        print("✅ 학습 완료")
        
        # 모델 평가
        self.evaluate_model(X_test_tensor.to(self.device), y_test_tensor.to(self.device))
        
        return self.model
    
    def evaluate_model(self, X_test, y_test):
        """모델 평가"""
        print("\n📊 모델 평가 시작...")
        
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_test).cpu()
            predictions = outputs.argmax(1)
        
        # 정확도 계산
        accuracy = accuracy_score(y_test.cpu(), predictions)
        print(f"🎯 테스트 정확도: {accuracy:.4f}")
        
        return accuracy
    
    def plot_training_history(self):
        """학습 과정 시각화"""
        if not self.training_history:
            print("학습 기록이 없습니다.")
            return
        
        epochs = [h['epoch'] for h in self.training_history]
        losses = [h['loss'] for h in self.training_history]
        accuracies = [h['accuracy'] for h in self.training_history]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        
        # 손실 그래프
        ax1.plot(epochs, losses, 'b-', linewidth=2)
        ax1.set_title('학습 손실')
        ax1.set_xlabel('에포크')
        ax1.set_ylabel('손실')
        ax1.grid(True, alpha=0.3)
        
        # 정확도 그래프
        ax2.plot(epochs, accuracies, 'r-', linewidth=2)
        ax2.set_title('학습 정확도')
        ax2.set_xlabel('에포크')
        ax2.set_ylabel('정확도')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# 모델 트레이너 인스턴스 생성
trainer = ModelTrainer(emotion_config, device)

# 학습 실행 예시 (전처리된 데이터가 있을 때)
if 'X_train' in locals() and X_train is not None:
    print("🎯 모델 학습 시작...")
    model = trainer.train_model(X_train, y_train, X_test, y_test, epochs=80)
    print("✅ 모델 학습 완료!")
else:
    print("⚠️ 먼저 데이터를 전처리해주세요.")
    print("실행 순서:")
    print("1. 데이터 로딩")
    print("2. 데이터 전처리") 
    print("3. 모델 학습")

🎯 모델 학습 시작...
🚀 모델 학습 시작...


학습 진행:   0%|          | 0/80 [00:00<?, ?it/s]

📈 학습 시작 - 80 에포크, 배치 크기: 256


학습 진행:  25%|██▌       | 20/80 [01:21<03:52,  3.88s/it]

에포크 20/80 - 손실: 1.5990, 정확도: 0.864


학습 진행:  50%|█████     | 40/80 [02:45<03:09,  4.74s/it]

에포크 40/80 - 손실: 1.5924, 정확도: 0.870


학습 진행:  52%|█████▎    | 42/80 [02:57<03:26,  5.43s/it]

In [18]:
# ## 8. 학습 결과 시각화

# In[8]:
# 학습 과정 시각화
if hasattr(trainer, 'training_history') and trainer.training_history:
    trainer.plot_training_history()
else:
    print("⚠️ 학습 기록이 없습니다. 먼저 모델을 학습해주세요.")

⚠️ 학습 기록이 없습니다. 먼저 모델을 학습해주세요.


In [ ]:
# In[9]:
class ModelManager:
    """모델 저장/로드 관리 클래스"""
    
    def __init__(self):
        pass
    
    def save_model(self, model, preprocessor, emotion_config, file_path):
        """모델 및 전처리기 저장"""
        try:
            # 디렉토리 생성
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            
            save_data = {
                'model_state_dict': model.state_dict(),
                'scaler': preprocessor.scaler,
                'label_encoder': preprocessor.label_encoder,
                'imputation_means': preprocessor.imputation_means,
                'common_features': preprocessor.common_features,
                'emotions_config': emotion_config.emotions,
                'korean_emotion_map': emotion_config.korean_emotion_map,
                'input_dim': len(preprocessor.common_features) if preprocessor.common_features else 0,
                'num_classes': len(preprocessor.label_encoder.classes_) if hasattr(preprocessor.label_encoder, 'classes_') else 0
            }
            
            torch.save(save_data, file_path)
            print(f"✅ 모델 저장 완료: {file_path}")
            return True
            
        except Exception as e:
            print(f"❌ 모델 저장 실패: {e}")
            return False
    
    def load_model(self, file_path, device):
        """모델 및 전처리기 로드"""
        try:
            if not os.path.exists(file_path):
                print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
                return None, None, None
            
            checkpoint = torch.load(file_path, map_location=device, weights_only=False)
            
            # 모델 재생성
            input_dim = checkpoint['input_dim']
            num_classes = checkpoint['num_classes']
            
            model = MusicEmotionClassifier(
                input_dim=input_dim,
                num_classes=num_classes
            ).to(device)
            
            model.load_state_dict(checkpoint['model_state_dict'])
            model.eval()
            
            # 전처리기 복원
            preprocessor_restored = DataPreprocessor()
            preprocessor_restored.scaler = checkpoint['scaler']
            preprocessor_restored.label_encoder = checkpoint['label_encoder']
            preprocessor_restored.imputation_means = checkpoint['imputation_means']
            preprocessor_restored.common_features = checkpoint['common_features']
            
            # 감정 설정 복원
            emotion_config_restored = EmotionConfig()
            if 'emotions_config' in checkpoint:
                emotion_config_restored.emotions = checkpoint['emotions_config']
            if 'korean_emotion_map' in checkpoint:
                emotion_config_restored.korean_emotion_map = checkpoint['korean_emotion_map']
            
            print(f"✅ 모델 로드 완료: {file_path}")
            return model, preprocessor_restored, emotion_config_restored
            
        except Exception as e:
            print(f"❌ 모델 로드 실패: {e}")
            return None, None, None

# 모델 매니저 인스턴스 생성
model_manager = ModelManager()

# 모델 저장 (학습된 모델이 있을 때)
MODEL_SAVE_PATH = "models/emotion_classifier.pth"

if hasattr(trainer, 'model') and trainer.model is not None:
    success = model_manager.save_model(
        trainer.model, preprocessor, emotion_config, MODEL_SAVE_PATH
    )
    if success:
        print(f"💾 모델이 저장되었습니다: {MODEL_SAVE_PATH}")
else:
    print("⚠️ 저장할 모델이 없습니다. 먼저 모델을 학습해주세요.")


🎯 추천 시스템 테스트...

[Joy 감정 추천 곡]
  🎵 Your Love Is All I Need - Original Mix - Liz Torres (유사도: 0.990)
  🎵 Tiroler Bua (Wellerman - Hüttenstyle) - Matty Valentino;DJ MOX;Ivan Fillini;Toby Tyrol;Xandl (유사도: 0.989)
  🎵 半點心 - Grasshopper (유사도: 0.988)

[Sadness 감정 추천 곡]
  🎵 Acompáñame - Alberto Vazquez (유사도: 0.991)
  🎵 O Holy Night - Top of the Bus (유사도: 0.985)
  🎵 Komm Zigany (auch ich war einst ein feiner Csárdáskavalier) - Emmerich Kálmán;Peter Alexander (유사도: 0.982)

[Anger 감정 추천 곡]
  🎵 Point Of No Return - Pappenheimer (유사도: 0.990)
  🎵 Juggernaut - Smile on the Sinner (유사도: 0.987)
  🎵 Blame Myself - ILLENIUM and Virtual Riot Remix - ILLENIUM;Tori Kelly;Virtual Riot (유사도: 0.986)

[Fear 감정 추천 곡]
  🎵 Amnesia - Gavin Mikhail (유사도: 0.996)
  🎵 Atlantis (Sped Up) - fenekot (유사도: 0.987)
  🎵 難破船 - Akina Nakamori (유사도: 0.986)

[Disgust 감정 추천 곡]
  🎵 Cordeiro e Leão - Ao Vivo - Central 3;Pevê Brito;Drops INA (유사도: 0.985)
  🎵 Tere Bin - Raghav Chaitanya (유사도: 0.983)
  🎵 Someday At Christmas - Stevie

In [ ]:
# ## 10. 음악 추천 시스템

# In[10]:
class MusicRecommendationSystem:
    """음악 추천 시스템"""
    
    def __init__(self, model, preprocessor, emotion_config, device):
        self.model = model
        self.preprocessor = preprocessor
        self.emotion_config = emotion_config
        self.device = device
    
    def recommend_by_valence(self, emotion, n, data_df):
        """Valence 기반 추천"""
        if emotion not in self.emotion_config.emotions:
            return pd.DataFrame()
        
        valence_range = self.emotion_config.emotions[emotion]['valence']
        vmin, vmax = valence_range
        
        filtered_df = data_df[
            (data_df['valence'] >= vmin) & (data_df['valence'] < vmax)
        ]
        
        if len(filtered_df) == 0:
            return pd.DataFrame()
        
        return filtered_df.sample(n=min(n, len(filtered_df)))
    
    def recommend_by_similarity(self, emotion, n, data_df, train_df):
        """코사인 유사도 기반 추천"""
        features = self.preprocessor.get_feature_names()
        if not features:
            return pd.DataFrame()
        
        # 데이터 전처리
        df_features = data_df[features].fillna(self.preprocessor.imputation_means)
        df_features = df_features.replace([np.inf, -np.inf], self.preprocessor.imputation_means.mean())
        
        # 스케일링
        scaled_features = self.preprocessor.scaler.transform(df_features.values)
        
        # 감정 프로필 계산
        emotion_data = train_df[train_df['emotion_cat'] == emotion]
        if emotion_data.empty:
            return pd.DataFrame()
        
        profile_raw = emotion_data[features].mean()
        profile_filled = profile_raw.fillna(self.preprocessor.imputation_means)
        profile_scaled = self.preprocessor.scaler.transform(profile_filled.values.reshape(1, -1))
        
        # 유사도 계산
        similarities = cosine_similarity(profile_scaled, scaled_features)[0]
        
        # 결과 정렬
        data_df_copy = data_df.copy()
        data_df_copy['similarity'] = similarities
        
        return data_df_copy.nlargest(n, 'similarity')
    
    def recommend_by_neural_network(self, emotion, n, data_df):
        """신경망 기반 추천"""
        features = self.preprocessor.get_feature_names()
        if not features:
            return pd.DataFrame()
        
        # 데이터 전처리
        df_features = data_df[features].fillna(self.preprocessor.imputation_means)
        df_features = df_features.replace([np.inf, -np.inf], self.preprocessor.imputation_means.mean())
        
        # 스케일링
        scaled_features = self.preprocessor.scaler.transform(df_features.values)
        
        # 예측
        self.model.eval()
        with torch.no_grad():
            features_tensor = torch.FloatTensor(scaled_features).to(self.device)
            probabilities = self.model(features_tensor).cpu().numpy()
        
        # 감정 인덱스 찾기
        try:
            emotion_idx = self.preprocessor.label_encoder.transform([emotion])[0]
        except ValueError:
            emotion_idx = 0
        
        # 결과 정렬
        data_df_copy = data_df.copy()
        data_df_copy['probability'] = probabilities[:, emotion_idx]
        
        return data_df_copy.nlargest(n, 'probability')
    
    def recommend_hybrid(self, target_emotion, n, data_df, train_df=None):
        """하이브리드 추천 (여러 방법 결합)"""
        recommendations = []
        fetch_count = max(n * 2, 10)  # 더 많이 가져와서 중복 제거
        
        methods = [
            ('valence', lambda: self.recommend_by_valence(target_emotion, fetch_count, data_df)),
            ('neural', lambda: self.recommend_by_neural_network(target_emotion, fetch_count, data_df))
        ]
        
        # 유사도 방법은 학습 데이터가 있을 때만
        if train_df is not None:
            methods.append(
                ('similarity', lambda: self.recommend_by_similarity(target_emotion, fetch_count, data_df, train_df))
            )
        
        for method_name, method_func in methods:
            try:
                result = method_func()
                if not result.empty:
                    recommendations.append(result)
            except Exception as e:
                print(f"⚠️ {method_name} 방법 실행 중 오류: {e}")
        
        if not recommendations:
            return pd.DataFrame()
        
        # 결합 및 중복 제거
        combined = pd.concat(recommendations, ignore_index=True)
        
        # track_name과 track_artist가 있는 경우에만 중복 제거
        if 'track_name' in combined.columns and 'track_artist' in combined.columns:
            return combined.drop_duplicates(subset=['track_name', 'track_artist']).head(n)
        else:
            return combined.drop_duplicates().head(n)
    
    def get_emotion_from_korean(self, korean_text):
        """한국어 텍스트에서 감정 추출"""
        emotion, keyword = self.emotion_config.map_korean_emotion(korean_text)
        if emotion:
            emotion_label = self.emotion_config.get_emotion_label(emotion)
            print(f"'{korean_text}'에서 '{keyword}' 키워드로 '{emotion_label}' 감정을 감지했습니다.")
            return emotion
        else:
            print(f"'{korean_text}'에서 감정을 감지할 수 없어 '평온' 감정으로 추천합니다.")
            return 'calm'

print("✅ 음악 추천 시스템 클래스 정의 완료")


🔮 감정 예측 테스트

1. 기쁜 음악 특성으로 예측:
예측 감정: Joy
신뢰도: 1.000

2. 슬픈 음악 특성으로 예측:
예측 감정: Sadness
신뢰도: 0.910

3. 가사로 감정 예측:
예측 감정: Joy
신뢰도: 0.419

✨ 모든 테스트 완료!


In [ ]:
# In[11]:
class EmotionPredictor:
    """감정 예측 시스템"""
    
    def __init__(self, model, preprocessor, emotion_config, device):
        self.model = model
        self.preprocessor = preprocessor
        self.emotion_config = emotion_config
        self.device = device
    
    def predict_emotion(self, audio_features):
        """음향 특성으로부터 감정 예측"""
        features = self.preprocessor.get_feature_names()
        
        if len(audio_features) != len(features):
            print(f"❌ 입력 특성 수가 맞지 않습니다. 예상: {len(features)}, 입력: {len(audio_features)}")
            return None, None, None
        
        # 특성 전처리
        features_series = pd.Series(audio_features, index=features)
        features_series = features_series.replace([np.inf, -np.inf], np.nan)
        
        # 결측값 처리
        features_filled = features_series.fillna(self.preprocessor.imputation_means)
        
        # 스케일링
        features_scaled = self.preprocessor.scaler.transform(features_filled.values.reshape(1, -1))
        
        # 예측
        self.model.eval()
        with torch.no_grad():
            features_tensor = torch.FloatTensor(features_scaled).to(self.device)
            probabilities = self.model(features_tensor).cpu().numpy()[0]
        
        # 최고 확률 감정
        predicted_idx = probabilities.argmax()
        predicted_emotion = self.preprocessor.label_encoder.inverse_transform([predicted_idx])[0]
        confidence = probabilities[predicted_idx]
        
        # 전체 확률 분포
        emotion_classes = self.preprocessor.get_emotion_classes()
        emotion_labels = [self.emotion_config.get_emotion_label(e) for e in emotion_classes]
        
        probability_distribution = {
            emotion_labels[i]: prob for i, prob in enumerate(probabilities)
        }
        
        return predicted_emotion, confidence, probability_distribution
    
    def analyze_playlist(self, track_names, data_df):
        """플레이리스트 감정 분석"""
        if 'track_name' not in data_df.columns:
            print("❌ 트랙명 컬럼을 찾을 수 없습니다.")
            return None, None
        
        # 플레이리스트 트랙 찾기
        playlist_tracks = data_df[data_df['track_name'].isin(track_names)]
        
        if playlist_tracks.empty:
            print("❌ 플레이리스트의 트랙을 데이터셋에서 찾을 수 없습니다.")
            return None, None
        
        print(f"📊 플레이리스트에서 {len(playlist_tracks)} 트랙을 찾았습니다.")
        
        # 평균 Valence 계산
        avg_valence = playlist_tracks['valence'].mean() if 'valence' in playlist_tracks.columns else 0
        
        # 가장 빈번한 감정
        if 'emotion_cat' in playlist_tracks.columns:
            emotion_counts = playlist_tracks['emotion_cat'].value_counts()
            most_common_emotion = emotion_counts.index[0]
            emotion_label = self.emotion_config.get_emotion_label(most_common_emotion)
        else:
            emotion_label = '알 수 없음'
        
        return emotion_label, avg_valence

print("✅ 감정 예측 시스템 클래스 정의 완료")


💾 결과 저장...
✅ 결과 파일 저장 완료!
- processed_train_data.csv: 처리된 훈련 데이터
- processed_test_data.csv: 처리된 테스트 데이터
- emotion_recommendations.csv: 감정별 추천 곡

📥 결과 파일 다운로드:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ## 12. 시스템 통합 및 사용 예시

# In[12]:
# 시스템 통합 클래스
class IntegratedMusicSystem:
    """통합 음악 감정 시스템"""
    
    def __init__(self):
        self.model = None
        self.preprocessor = None
        self.emotion_config = None
        self.recommendation_system = None
        self.emotion_predictor = None
        self.data_loader = None
        self.device = device
    
    def initialize_from_trained_components(self, model, preprocessor, emotion_config, data_loader):
        """학습된 컴포넌트로 초기화"""
        self.model = model
        self.preprocessor = preprocessor
        self.emotion_config = emotion_config
        self.data_loader = data_loader
        
        self.recommendation_system = MusicRecommendationSystem(
            model, preprocessor, emotion_config, self.device
        )
        self.emotion_predictor = EmotionPredictor(
            model, preprocessor, emotion_config, self.device
        )
        
        print("✅ 통합 시스템 초기화 완료")
    
    def load_from_saved_model(self, model_path):
        """저장된 모델에서 로드"""
        model_manager = ModelManager()
        model, preprocessor, emotion_config = model_manager.load_model(model_path, self.device)
        
        if model is not None:
            self.model = model
            self.preprocessor = preprocessor
            self.emotion_config = emotion_config
            
            self.recommendation_system = MusicRecommendationSystem(
                model, preprocessor, emotion_config, self.device
            )
            self.emotion_predictor = EmotionPredictor(
                model, preprocessor, emotion_config, self.device
            )
            
            print("✅ 저장된 모델에서 시스템 로드 완료")
            return True
        else:
            print("❌ 모델 로드 실패")
            return False
    
    def recommend_music(self, emotion_text, n=5, method='hybrid', data_source='test'):
        """음악 추천"""
        if not self.recommendation_system:
            print("❌ 추천 시스템이 초기화되지 않았습니다.")
            return pd.DataFrame()
        
        # 감정 매핑
        target_emotion = self.recommendation_system.get_emotion_from_korean(emotion_text)
        
        # 데이터 선택
        if data_source == 'test' and hasattr(self.data_loader, 'test_df'):
            data_df = self.data_loader.test_df
            train_df = self.data_loader.train_df if hasattr(self.data_loader, 'train_df') else None
        elif data_source == 'train' and hasattr(self.data_loader, 'train_df'):
            data_df = self.data_loader.train_df
            train_df = self.data_loader.train_df
        else:
            print(f"❌ {data_source} 데이터를 찾을 수 없습니다.")
            return pd.DataFrame()
        
        # 추천 실행
        if method == 'hybrid':
            return self.recommendation_system.recommend_hybrid(target_emotion, n, data_df, train_df)
        elif method == 'valence':
            return self.recommendation_system.recommend_by_valence(target_emotion, n, data_df)
        elif method == 'neural':
            return self.recommendation_system.recommend_by_neural_network(target_emotion, n, data_df)
        elif method == 'similarity' and train_df is not None:
            return self.recommendation_system.recommend_by_similarity(target_emotion, n, data_df, train_df)
        else:
            print(f"❌ 지원하지 않는 추천 방법입니다: {method}")
            return pd.DataFrame()
    
    def predict_music_emotion(self, audio_features):
        """음악 감정 예측"""
        if not self.emotion_predictor:
            print("❌ 감정 예측 시스템이 초기화되지 않았습니다.")
            return None, None, None
        
        return self.emotion_predictor.predict_emotion(audio_features)
    
    def analyze_playlist_emotion(self, track_names, data_source='test'):
        """플레이리스트 감정 분석"""
        if not self.emotion_predictor:
            print("❌ 감정 예측 시스템이 초기화되지 않았습니다.")
            return None, None
        
        # 데이터 선택
        if data_source == 'test' and hasattr(self.data_loader, 'test_df'):
            data_df = self.data_loader.test_df
        elif data_source == 'train' and hasattr(self.data_loader, 'train_df'):
            data_df = self.data_loader.train_df
        else:
            print(f"❌ {data_source} 데이터를 찾을 수 없습니다.")
            return None, None
        
        return self.emotion_predictor.analyze_playlist(track_names, data_df)
    
    def get_system_status(self):
        """시스템 상태 확인"""
        status = {
            '모델 로드됨': self.model is not None,
            '전처리기 준비됨': self.preprocessor is not None,
            '감정 설정 로드됨': self.emotion_config is not None,
            '추천 시스템 준비됨': self.recommendation_system is not None,
            '예측 시스템 준비됨': self.emotion_predictor is not None,
            '데이터 로드됨': self.data_loader is not None,
        }
        
        return status

# 통합 시스템 인스턴스 생성
integrated_system = IntegratedMusicSystem()

# 학습된 컴포넌트가 있다면 초기화
if all(var in locals() for var in ['trainer', 'preprocessor', 'emotion_config', 'data_loader']):
    if hasattr(trainer, 'model') and trainer.model is not None:
        integrated_system.initialize_from_trained_components(
            trainer.model, preprocessor, emotion_config, data_loader
        )
    else:
        print("⚠️ 학습된 모델이 없습니다.")
else:
    print("⚠️ 필요한 컴포넌트가 준비되지 않았습니다.")

In [ ]:
# ## 13. 사용 예시 및 데모

# In[13]:
def run_recommendation_demo(system):
    """추천 시스템 데모"""
    print("🎵 음악 추천 데모")
    print("=" * 50)
    
    # 시스템 상태 확인
    status = system.get_system_status()
    if not all(status.values()):
        print("❌ 시스템이 완전히 준비되지 않았습니다.")
        print("시스템 상태:")
        for key, value in status.items():
            status_icon = "✅" if value else "❌"
            print(f"   {status_icon} {key}")
        return
    
    # 다양한 감정으로 추천 테스트
    test_emotions = [
        "오늘 정말 기쁜 하루였어!",
        "비 오는 날이라 우울해",
        "시험 때문에 스트레스 받아",
        "집에서 편안하게 쉬고 싶어"
    ]
    
    for emotion_text in test_emotions:
        print(f"\n🎯 감정: '{emotion_text}'")
        
        recommendations = system.recommend_music(emotion_text, n=3, method='hybrid')
        
        if not recommendations.empty:
            print("📀 추천 곡목:")
            display_columns = []
            
            if 'track_name' in recommendations.columns:
                display_columns.append('track_name')
            if 'track_artist' in recommendations.columns:
                display_columns.append('track_artist')
            if 'valence' in recommendations.columns:
                display_columns.append('valence')
            if 'emotion_cat' in recommendations.columns:
                display_columns.append('emotion_cat')
            
            if display_columns:
                for idx, row in recommendations[display_columns].head(3).iterrows():
                    track_info = []
                    if 'track_name' in row:
                        track_info.append(f"🎵 {row['track_name']}")
                    if 'track_artist' in row:
                        track_info.append(f"- {row['track_artist']}")
                    if 'valence' in row:
                        track_info.append(f"(valence: {row['valence']:.3f})")
                    
                    print(f"   {' '.join(track_info)}")
            else:
                print("   곡 정보를 표시할 수 없습니다.")
        else:
            print("   ❌ 추천곡을 찾을 수 없습니다.")
        
        print("-" * 40)

def run_prediction_demo(system):
    """감정 예측 데모"""
    print("🎭 감정 예측 데모")
    print("=" * 50)
    
    # 시스템 상태 확인
    status = system.get_system_status()
    if not status['예측 시스템 준비됨']:
        print("❌ 예측 시스템이 준비되지 않았습니다.")
        return
    
    # 샘플 음향 특성들
    sample_tracks = [
        {
            'name': '밝고 경쾌한 팝송',
            'features': [0.85, 0.78, 0.82, 0.15, 0.02, 0.25, 0.08, -4.5, 125.0]
        },
        {
            'name': '감성적인 발라드',
            'features': [0.15, 0.25, 0.35, 0.85, 0.05, 0.12, 0.04, -12.0, 65.0]
        },
        {
            'name': '격렬한 록 음악',
            'features': [0.25, 0.92, 0.65, 0.08, 0.01, 0.35, 0.15, -2.8, 145.0]
        }
    ]
    
    feature_names = system.preprocessor.get_feature_names()
    print(f"🔍 사용 특성: {feature_names}")
    print()
    
    for track in sample_tracks:
        print(f"🎼 {track['name']}")
        
        emotion, confidence, distribution = system.predict_music_emotion(track['features'])
        
        if emotion:
            emotion_label = system.emotion_config.get_emotion_label(emotion)
            print(f"   🎭 예측 감정: {emotion_label} (신뢰도: {confidence:.3f})")
            
            # 상위 3개 감정 확률
            sorted_emotions = sorted(distribution.items(), key=lambda x: x[1], reverse=True)[:3]
            print("   📊 상위 3개 감정:")
            for i, (emotion_name, prob) in enumerate(sorted_emotions, 1):
                bar = "█" * int(prob * 15)
                print(f"      {i}. {emotion_name}: {prob:.3f} {bar}")
        else:
            print("   ❌ 감정 예측 실패")
        
        print("-" * 50)

# 데모 실행
print("🚀 통합 시스템 데모 실행")

if integrated_system.get_system_status()['모델 로드됨']:
    run_recommendation_demo(integrated_system)
    run_prediction_demo(integrated_system)
else:
    print("⚠️ 먼저 모델을 학습하거나 저장된 모델을 로드해주세요.")


In [ ]:
# ## 14. 데이터 시각화

# In[14]:
def plot_emotion_distribution(data_df, emotion_config, title="감정 분포"):
    """감정 분포 시각화"""
    if 'emotion_cat' not in data_df.columns:
        print("❌ emotion_cat 컬럼이 없습니다.")
        return
    
    plt.figure(figsize=(12, 6))
    
    # 감정 분포 계산
    emotion_counts = data_df['emotion_cat'].value_counts()
    
    # 한국어 레이블로 변환
    korean_labels = [emotion_config.get_emotion_label(emotion) for emotion in emotion_counts.index]
    
    # 막대 그래프
    plt.subplot(1, 2, 1)
    bars = plt.bar(korean_labels, emotion_counts.values, color=plt.cm.Set3(np.arange(len(korean_labels))))
    plt.title(f'{title} - 곡 수')
    plt.xlabel('감정')
    plt.ylabel('곡 수')
    plt.xticks(rotation=45)
    
    # 값 표시
    for bar, count in zip(bars, emotion_counts.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(emotion_counts.values)*0.01,
                f'{count:,}', ha='center', va='bottom')
    
    # 파이 차트
    plt.subplot(1, 2, 2)
    plt.pie(emotion_counts.values, labels=korean_labels, autopct='%1.1f%%', 
            colors=plt.cm.Set3(np.arange(len(korean_labels))))
    plt.title(f'{title} - 비율')
    
    plt.tight_layout()
    plt.show()

def plot_valence_energy_scatter(data_df, emotion_config, title="Valence-Energy 분포"):
    """Valence-Energy 산점도"""
    if not all(col in data_df.columns for col in ['valence', 'energy', 'emotion_cat']):
        print("❌ 필요한 컬럼이 없습니다.")
        return
    
    plt.figure(figsize=(12, 8))
    
    # 감정별로 다른 색상
    emotions = data_df['emotion_cat'].unique()
    colors = plt.cm.Set3(np.linspace(0, 1, len(emotions)))
    
    for emotion, color in zip(emotions, colors):
        emotion_data = data_df[data_df['emotion_cat'] == emotion]
        label = emotion_config.get_emotion_label(emotion)
        
        plt.scatter(emotion_data['valence'], emotion_data['energy'], 
                   alpha=0.6, c=[color], label=label, s=30)
    
    plt.xlabel('Valence (긍정성)', fontsize=12)
    plt.ylabel('Energy (에너지)', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# 시각화 실행
if hasattr(data_loader, 'train_df') and data_loader.train_df is not None:
    print("📊 학습 데이터 시각화")
    plot_emotion_distribution(data_loader.train_df, emotion_config, "학습 데이터 감정 분포")
    plot_valence_energy_scatter(data_loader.train_df, emotion_config, "학습 데이터 Valence-Energy 분포")

if hasattr(data_loader, 'test_df') and data_loader.test_df is not None:
    print("📊 테스트 데이터 시각화")
    plot_emotion_distribution(data_loader.test_df, emotion_config, "테스트 데이터 감정 분포")



In [ ]:
# ## 15. 시스템 상태 점검 및 문제 해결

# In[15]:
def comprehensive_system_check():
    """종합 시스템 점검"""
    print("🔧 종합 시스템 상태 점검")
    print("=" * 60)
    
    # 1. 데이터 로드 상태
    print("📊 1. 데이터 상태")
    print("-" * 30)
    
    train_status = hasattr(data_loader, 'train_df') and data_loader.train_df is not None
    test_status = hasattr(data_loader, 'test_df') and data_loader.test_df is not None
    
    if train_status:
        print(f"✅ 학습 데이터: {len(data_loader.train_df):,} 트랙")
        print(f"   컬럼: {list(data_loader.train_df.columns)}")
    else:
        print("❌ 학습 데이터 없음")
        print("   해결방법: data_loader.load_train_data('경로') 실행")
    
    if test_status:
        print(f"✅ 테스트 데이터: {len(data_loader.test_df):,} 트랙")
        print(f"   컬럼: {list(data_loader.test_df.columns)}")
    else:
        print("❌ 테스트 데이터 없음")
        print("   해결방법: data_loader.load_test_data('경로') 실행")
    
    # 2. 모델 상태
    print(f"\n🤖 2. 모델 상태")
    print("-" * 30)
    
    model_status = hasattr(trainer, 'model') and trainer.model is not None
    if model_status:
        print("✅ 모델 학습 완료")
        if hasattr(preprocessor, 'common_features'):
            print(f"   입력 특성: {len(preprocessor.common_features)}개")
            print(f"   감정 클래스: {len(preprocessor.get_emotion_classes())}개")
    else:
        print("❌ 모델 미학습")
        print("   해결방법: trainer.train_model() 실행")
    
    # 3. 통합 시스템 상태
    print(f"\n🎵 3. 통합 시스템 상태")
    print("-" * 30)
    
    if 'integrated_system' in locals():
        system_status = integrated_system.get_system_status()
        for component, status in system_status.items():
            icon = "✅" if status else "❌"
            print(f"   {icon} {component}")
    else:
        print("❌ 통합 시스템 미초기화")
    
    # 4. 권장 사항
    print(f"\n💡 4. 권장 사항")
    print("-" * 30)
    
    recommendations = []
    
    if not train_status or not test_status:
        recommendations.append("데이터 파일 경로를 확인하고 올바른 데이터셋을 로드하세요")
    
    if not model_status:
        recommendations.append("모델을 학습하거나 기존 저장된 모델을 로드하세요")
    
    if train_status and test_status and model_status:
        recommendations.append("모든 시스템이 정상입니다! 추천 및 예측 기능을 사용할 수 있습니다")
    
    for i, rec in enumerate(recommendations, 1):
        print(f"   {i}. {rec}")
    
    # 5. 빠른 시작 가이드
    print(f"\n🚀 5. 빠른 시작 가이드")
    print("-" * 30)
    print("1. 데이터 로드:")
    print("   data_loader.load_train_data('경로/spotify_tracks_dataset.csv')")
    print("   data_loader.load_test_data('경로/Spotify_Youtube.csv')")
    print()
    print("2. 데이터 전처리:")
    print("   X_train, y_train, X_test, y_test = preprocessor.prepare_data(...)")
    print()
    print("3. 모델 학습:")
    print("   model = trainer.train_model(X_train, y_train, X_test, y_test)")
    print()
    print("4. 시스템 사용:")
    print("   integrated_system.recommend_music('기쁜 기분이야!', n=5)")
    print("   integrated_system.predict_music_emotion([0.8, 0.7, ...])")

# 시스템 점검 실행
comprehensive_system_check()

In [ ]:
# ## 16. 사용자 정의 함수들

# In[16]:
def quick_recommendation(emotion_text, n=5):
    """빠른 추천 함수"""
    if 'integrated_system' not in locals() or not integrated_system.get_system_status()['모델 로드됨']:
        print("❌ 시스템이 준비되지 않았습니다.")
        return None
    
    return integrated_system.recommend_music(emotion_text, n=n, method='hybrid')

def quick_prediction(audio_features):
    """빠른 예측 함수"""
    if 'integrated_system' not in locals() or not integrated_system.get_system_status()['모델 로드됨']:
        print("❌ 시스템이 준비되지 않았습니다.")
        return None, None, None
    
    return integrated_system.predict_music_emotion(audio_features)

def analyze_my_playlist(track_names):
    """내 플레이리스트 분석"""
    if 'integrated_system' not in locals() or not integrated_system.get_system_status()['모델 로드됨']:
        print("❌ 시스템이 준비되지 않았습니다.")
        return None, None
    
    return integrated_system.analyze_playlist_emotion(track_names)

def save_current_model(save_path="models/my_emotion_model.pth"):
    """현재 모델 저장"""
    if 'trainer' not in locals() or not hasattr(trainer, 'model') or trainer.model is None:
        print("❌ 저장할 모델이 없습니다.")
        return False
    
    return model_manager.save_model(trainer.model, preprocessor, emotion_config, save_path)

def load_saved_model(model_path):
    """저장된 모델 로드"""
    global integrated_system
    integrated_system = IntegratedMusicSystem()
    return integrated_system.load_from_saved_model(model_path)

# 편의 함수들 등록
print("✅ 편의 함수들이 준비되었습니다:")
print("   - quick_recommendation('기분 설명', n=5)")
print("   - quick_prediction([특성값들])")
print("   - analyze_my_playlist(['곡명1', '곡명2', ...])")
print("   - save_current_model('경로')")
print("   - load_saved_model('경로')")

In [ ]:
# In[17]:
def display_usage_guide():
    """사용법 가이드 표시"""
    guide = """
    🎵 Spotify 감정 기반 음악 추천 시스템 사용 가이드
    ===============================================
    
    📁 1. 데이터 준비
    ----------------
    • Spotify Tracks Dataset을 다운로드하여 'data/' 폴더에 저장
    • Spotify YouTube Dataset을 다운로드하여 'data/' 폴더에 저장
    
    🚀 2. 기본 사용법
    -----------------
    # 데이터 로드
    data_loader.load_train_data('data/spotify_tracks_dataset.csv')
    data_loader.load_test_data('data/Spotify_Youtube.csv')
    
    # 데이터 전처리
    common_features = data_loader.get_common_features()
    X_train, y_train, X_test, y_test = preprocessor.prepare_data(
        data_loader.train_df, data_loader.test_df, common_features
    )
    
    # 모델 학습
    model = trainer.train_model(X_train, y_train, X_test, y_test, epochs=80)
    
    # 시스템 초기화
    integrated_system.initialize_from_trained_components(
        trainer.model, preprocessor, emotion_config, data_loader
    )
    
    🎯 3. 주요 기능
    ---------------
    # 음악 추천
    recommendations = integrated_system.recommend_music('기쁜 기분이야!', n=5)
    
    # 감정 예측
    emotion, confidence, distribution = integrated_system.predict_music_emotion(
        [0.8, 0.7, 0.6, 0.3, 0.1, 0.2, 0.1, -6.0, 110.0]
    )
    
    # 플레이리스트 분석
    emotion, valence = integrated_system.analyze_playlist_emotion(
        ['Shape of You', 'Blinding Lights', 'Bohemian Rhapsody']
    )
    
    💾 4. 모델 저장/로드
    -------------------
    # 저장
    model_manager.save_model(trainer.model, preprocessor, emotion_config, 'models/my_model.pth')
    
    # 로드
    integrated_system.load_from_saved_model('models/my_model.pth')
    
    🎭 5. 지원하는 감정
    ------------------
    • 기쁨이 (joy): 기쁨, 환희, 즐거움, 행복, 신남 등
    • 슬픔이 (sadness): 슬픔, 우울, 좌절, 외로움 등
    • 버럭이 (anger): 화남, 분노, 짜증, 열받음 등
    • 소심이 (anxiety): 불안, 걱정, 초조, 스트레스 등
    • 따분이 (boredom): 지루함, 심심함, 무기력 등
    • 까칠이 (grumpy): 까칠함, 예민함, 불쾌감 등
    • 불안이 (fear): 두려움, 공포, 무서움 등
    • 당황이 (embarrassment): 당황, 어색함, 민망함 등
    • 부럽이 (envy): 부러움, 질투 등
    • 평온 (calm): 평온, 차분함, 안정감 등
    
    🔧 6. 문제 해결
    ---------------
    • 시스템 상태 확인: comprehensive_system_check()
    • 데이터 시각화: plot_emotion_distribution(), plot_valence_energy_scatter()
    • 학습 과정 확인: trainer.plot_training_history()
    
    📖 7. 편의 함수
    ---------------
    • quick_recommendation('감정 설명')
    • quick_prediction([특성값들])
    • analyze_my_playlist(['곡명들'])
    • save_current_model('경로')
    • load_saved_model('경로')
    """
    
    print(guide)

# 사용법 가이드 표시
display_usage_guide()

In [ ]:
# ## 18. 테스트 및 검증

# In[18]:
def run_comprehensive_test():
    """종합 테스트 실행"""
    print("🧪 종합 시스템 테스트 시작")
    print("=" * 50)
    
    test_results = {
        'data_loading': False,
        'preprocessing': False,
        'model_training': False,
        'recommendation': False,
        'prediction': False,
        'playlist_analysis': False
    }
    
    # 1. 데이터 로딩 테스트
    print("1. 데이터 로딩 테스트...")
    if hasattr(data_loader, 'train_df') and data_loader.train_df is not None:
        if hasattr(data_loader, 'test_df') and data_loader.test_df is not None:
            test_results['data_loading'] = True
            print("   ✅ 데이터 로딩 성공")
        else:
            print("   ❌ 테스트 데이터 로딩 실패")
    else:
        print("   ❌ 학습 데이터 로딩 실패")
    
    # 2. 전처리 테스트
    print("2. 데이터 전처리 테스트...")
    if 'X_train' in locals() and X_train is not None:
        test_results['preprocessing'] = True
        print("   ✅ 데이터 전처리 성공")
    else:
        print("   ❌ 데이터 전처리 실패")
    
    # 3. 모델 학습 테스트
    print("3. 모델 학습 테스트...")
    if hasattr(trainer, 'model') and trainer.model is not None:
        test_results['model_training'] = True
        print("   ✅ 모델 학습 성공")
    else:
        print("   ❌ 모델 학습 실패")
    
    # 4. 추천 시스템 테스트
    print("4. 추천 시스템 테스트...")
    if 'integrated_system' in locals() and integrated_system.get_system_status()['추천 시스템 준비됨']:
        try:
            test_rec = integrated_system.recommend_music('기쁨', n=1)
            if not test_rec.empty:
                test_results['recommendation'] = True
                print("   ✅ 추천 시스템 성공")
            else:
                print("   ⚠️ 추천 시스템 작동하지만 결과 없음")
        except Exception as e:
            print(f"   ❌ 추천 시스템 오류: {e}")
    else:
        print("   ❌ 추천 시스템 미준비")
    
    # 5. 감정 예측 테스트
    print("5. 감정 예측 테스트...")
    if 'integrated_system' in locals() and integrated_system.get_system_status()['예측 시스템 준비됨']:
        try:
            test_features = [0.8, 0.7, 0.6, 0.3, 0.1, 0.2, 0.1, -6.0, 110.0]
            emotion, confidence, _ = integrated_system.predict_music_emotion(test_features)
            if emotion:
                test_results['prediction'] = True
                print(f"   ✅ 감정 예측 성공 (결과: {emotion})")
            else:
                print("   ❌ 감정 예측 실패")
        except Exception as e:
            print(f"   ❌ 감정 예측 오류: {e}")
    else:
        print("   ❌ 감정 예측 시스템 미준비")
    
    # 6. 플레이리스트 분석 테스트
    print("6. 플레이리스트 분석 테스트...")
    if 'integrated_system' in locals() and integrated_system.get_system_status()['예측 시스템 준비됨']:
        try:
            # 실제 존재할 가능성이 높은 곡명들로 테스트
            test_playlist = ['test_track_1', 'test_track_2']  # 실제 데이터에 맞게 수정 필요
            emotion, valence = integrated_system.analyze_playlist_emotion(test_playlist)
            if emotion is not None:
                test_results['playlist_analysis'] = True
                print(f"   ✅ 플레이리스트 분석 성공")
            else:
                print("   ⚠️ 플레이리스트 분석 - 해당 트랙을 찾을 수 없음 (정상)")
        except Exception as e:
            print(f"   ❌ 플레이리스트 분석 오류: {e}")
    else:
        print("   ❌ 플레이리스트 분석 시스템 미준비")
    
    # 결과 요약
    print(f"\n📊 테스트 결과 요약")
    print("-" * 30)
    
    passed_tests = sum(test_results.values())
    total_tests = len(test_results)
    
    for test_name, result in test_results.items():
        icon = "✅" if result else "❌"
        print(f"   {icon} {test_name.replace('_', ' ').title()}")
    
    print(f"\n🎯 전체 결과: {passed_tests}/{total_tests} 테스트 통과")
    
    if passed_tests == total_tests:
        print("🎉 모든 테스트 통과! 시스템이 완전히 작동합니다.")
    elif passed_tests >= total_tests * 0.7:
        print("👍 대부분의 기능이 정상 작동합니다.")
    else:
        print("⚠️ 일부 기능에 문제가 있습니다. 설정을 확인해주세요.")
    
    return test_results

# 종합 테스트 실행
test_results = run_comprehensive_test()

# ## 19. 마무리 및 정리

# In[19]:
print("🎵 Spotify 감정 기반 음악 추천 시스템")
print("=" * 60)
print("✅ 주피터 노트북 버전 설정 완료!")
print()
print("📋 주요 구성 요소:")
print("   • EmotionConfig: 감정 설정 및 매핑")
print("   • DataLoader: 데이터 로딩 및 전처리")
print("   • MusicEmotionClassifier: 신경망 모델")
print("   • ModelTrainer: 모델 학습")
print("   • MusicRecommendationSystem: 음악 추천")
print("   • EmotionPredictor: 감정 예측")
print("   • IntegratedMusicSystem: 통합 시스템")
print()
print("🎯 사용 가능한 기능:")
print("   • 감정별 음악 추천 (9가지 감정)")
print("   • 음향 특성 기반 감정 예측")
print("   • 플레이리스트 감정 분석")
print("   • 하이브리드 추천 알고리즘")
print("   • 모델 저장/로드")
print("   • 데이터 시각화")
print()
print("💡 빠른 시작:")
print("   1. 데이터 파일을 'data/' 폴더에 저장")
print("   2. 셀을 순차적으로 실행")
print("   3. quick_recommendation('기분 설명') 함수 사용")
print()
print("🔧 문제 해결:")
print("   • comprehensive_system_check() - 시스템 상태 확인")
print("   • display_usage_guide() - 상세 사용법")
print("   • run_comprehensive_test() - 전체 기능 테스트")
print()
print("🎉 음악과 감정의 아름다운 만남을 즐기세요!")

# 마지막 상태 확인
if 'integrated_system' in locals():
    final_status = integrated_system.get_system_status()
    ready_components = sum(final_status.values())
    total_components = len(final_status)
    
    if ready_components == total_components:
        print(f"\n🚀 시스템 완전 준비 완료! ({ready_components}/{total_components})")
        print("이제 모든 기능을 사용할 수 있습니다.")
    else:
        print(f"\n⚠️ 시스템 부분 준비 ({ready_components}/{total_components})")
        print("일부 기능을 사용하려면 추가 설정이 필요합니다.")
else:
    print("\n⚠️ 통합 시스템이 초기화되지 않았습니다.")
    print("위의 셀들을 순서대로 실행해주세요.")# ## 🎵 Spotify 감정 기반 음악 추천 시스템
# 